# Current Open Positions for Account DU9233079

This notebook retrieves and displays all current open positions from Interactive Brokers account DU9233079.
It includes position details, market values, unrealized P&L, and identifies vertical spreads.

In [31]:
import time
import threading
import pandas as pd
import random
import datetime
from dateutil import parser
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.contract import Contract
from ibapi.common import TickerId, BarData
from ibapi.order import Order
from ibapi.order_state import OrderState
from ibapi.execution import ExecutionFilter

# Configure pandas display options for better formatting
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 50)

In [32]:
class IBKRPositionApp(EWrapper, EClient):
    """Enhanced IBKR Position Analyzer for account DU9233079"""
    
    def __init__(self, target_account="DU9233079"):
        EClient.__init__(self, self)
        
        self.target_account = target_account
        
        # Store positions and market data
        self.positions = {}  # Dictionary to store position data
        self.options_data = {}  # Dictionary to store option market data
        self.vertical_spreads = []  # List to store identified vertical spreads
        self.req_id_to_option = {}  # Maps request IDs to option contracts
        self.next_req_id = 1
        self.position_end_received = False
        self.connection_problem = False
        
        # For account information
        self.account_info = {}
        self.account_summary_end_received = False
        
        # For portfolio updates
        self.portfolio = {}
        
        # Track historical data requests
        self.historical_data_requests = {}
        self.completed_hist_requests = set()
        
    def nextOrderId(self):
        """Get the next request ID and increment the counter"""
        id = self.next_req_id
        self.next_req_id += 1
        return id
    
    def error(self, reqId, errorCode, errorString, advancedOrderRejectJson="", *args):
        """Handle error messages from IB Gateway"""
        # Informational messages
        if errorCode in [2104, 2106, 2158]:
            print(f"Info: {errorString}")
            return
        
        # Market data errors - try alternative approach
        if errorCode == 321:
            print(f"Market data error for reqId {reqId}: {errorString}")
            # Cancel the problematic market data request
            self.cancelMktData(reqId)
            # Try historical data instead
            if reqId in self.req_id_to_option:
                self.request_historical_data_fallback(reqId)
            return
        
        # Actual errors
        if errorCode not in [2104, 2106, 2158, 321]:
            print(f"Error {errorCode}: {errorString}")
            if errorCode == 504:
                self.connection_problem = True
    
    def request_historical_data_fallback(self, original_req_id):
        """Request historical data when live market data fails"""
        if original_req_id in self.req_id_to_option:
            option_key = self.req_id_to_option[original_req_id]
            if option_key in self.positions:
                contract = self.positions[option_key]['contract']
                
                # Create new request ID for historical data
                hist_req_id = self.nextOrderId()
                self.historical_data_requests[hist_req_id] = option_key
                
                print(f"Requesting historical data for {contract.symbol} {contract.strike} {contract.right}")
                
                # Request 1 day of data to get recent closing price
                self.reqHistoricalData(
                    hist_req_id, 
                    contract, 
                    "", 
                    "1 D", 
                    "1 day", 
                    "MIDPOINT", 
                    1, 
                    1, 
                    False, 
                    []
                )
    
    def historicalData(self, reqId, bar):
        """Handle historical data response"""
        if reqId in self.historical_data_requests:
            option_key = self.historical_data_requests[reqId]
            if option_key not in self.options_data:
                self.options_data[option_key] = {}
            
            # Store the closing price from historical data
            self.options_data[option_key]['historical_close'] = float(bar.close)
            self.options_data[option_key]['historical_volume'] = float(bar.volume)
            print(f"Got historical data for {option_key}: Close=${bar.close:.2f}")
    
    def historicalDataEnd(self, reqId, start, end):
        """Handle end of historical data"""
        if reqId in self.historical_data_requests:
            self.completed_hist_requests.add(reqId)
            print(f"Historical data complete for reqId {reqId}")
    
    def isConnectedAndReady(self):
        """Check if we're connected and no connection problems reported"""
        return self.isConnected() and not self.connection_problem
    
    def connectTWS(self, host='127.0.0.1', port=4002, clientId=None):
        """Connect to IB Gateway"""
        if clientId is None:
            clientId = random.randint(1000, 9999)
            
        print(f"Connecting to IB Gateway at {host}:{port} with client ID: {clientId}")
        self.connect(host, port, clientId)
        
        # Start a thread to process messages
        thread = threading.Thread(target=self.run)
        thread.daemon = True
        thread.start()
        
        # Allow time for connection to establish
        time.sleep(3)
    
    def position(self, account, contract, position, avgCost):
        """Handle position data - filter for target account only"""
        # Only process positions for our target account
        if account != self.target_account:
            return
            
        if position != 0:  # Consider all non-zero positions
            print(f"Position received: {contract.symbol} ({contract.secType}), {position} @ ${avgCost:.2f}")
            
            # Create unique key for position
            if contract.secType == "OPT":
                key = (contract.symbol, contract.secType, contract.strike, contract.right, contract.lastTradeDateOrContractMonth)
            else:
                key = (contract.symbol, contract.secType, None, None, None)
                
            self.positions[key] = {
                'contract': contract,
                'position': float(position),  # Convert to float to avoid Decimal issues
                'avgCost': float(avgCost),    # Convert to float to avoid Decimal issues
                'account': account,
                'conId': contract.conId if hasattr(contract, 'conId') else None
            }
            
            # For options, try to get a more complete contract first
            if contract.secType == "OPT":
                # Create a more complete contract specification
                full_contract = Contract()
                full_contract.symbol = contract.symbol
                full_contract.secType = contract.secType
                full_contract.exchange = contract.exchange or "SMART"
                full_contract.currency = contract.currency or "USD"
                full_contract.strike = contract.strike
                full_contract.right = contract.right
                full_contract.lastTradeDateOrContractMonth = contract.lastTradeDateOrContractMonth
                full_contract.multiplier = contract.multiplier or "100"
                
                # Request market data with delay to avoid rate limiting
                req_id = self.nextOrderId()
                self.req_id_to_option[req_id] = key
                
                # Request market data with snapshot flag to avoid subscription
                self.reqMktData(req_id, full_contract, "", True, False, [])
            else:
                # For non-options, request market data normally
                req_id = self.nextOrderId()
                self.req_id_to_option[req_id] = key
                self.reqMktData(req_id, contract, "", True, False, [])
    
    def positionEnd(self):
        """Called when all positions have been reported"""
        print(f"Position data complete. Found {len(self.positions)} positions for account {self.target_account}")
        self.position_end_received = True
        self.find_vertical_spreads()
    
    def tickPrice(self, reqId, tickType, price, attrib):
        """Handle price updates"""
        if reqId in self.req_id_to_option:
            option_key = self.req_id_to_option[reqId]
            if option_key not in self.options_data:
                self.options_data[option_key] = {}
            
            # Store different price types and convert to float
            if tickType == 1:  # Bid
                self.options_data[option_key]['bid'] = float(price) if price > 0 else None
                if price > 0:
                    print(f"Got bid for {option_key}: ${price:.2f}")
            elif tickType == 2:  # Ask
                self.options_data[option_key]['ask'] = float(price) if price > 0 else None
                if price > 0:
                    print(f"Got ask for {option_key}: ${price:.2f}")
            elif tickType == 4:  # Last
                self.options_data[option_key]['last'] = float(price) if price > 0 else None
                if price > 0:
                    print(f"Got last for {option_key}: ${price:.2f}")
            elif tickType == 6:  # High
                self.options_data[option_key]['high'] = float(price) if price > 0 else None
            elif tickType == 7:  # Low
                self.options_data[option_key]['low'] = float(price) if price > 0 else None
            elif tickType == 9:  # Close
                self.options_data[option_key]['close'] = float(price) if price > 0 else None
                if price > 0:
                    print(f"Got close for {option_key}: ${price:.2f}")
    
    def accountSummary(self, reqId, account, tag, value, currency):
        """Handle account summary data"""
        if account == self.target_account:
            self.account_info[tag] = {'value': value, 'currency': currency}
    
    def accountSummaryEnd(self, reqId):
        """Called when account summary is complete"""
        self.account_summary_end_received = True
    
    def find_vertical_spreads(self):
        """Identify vertical spreads from the positions"""
        # Group options by symbol and expiry
        option_groups = {}
        for key, data in self.positions.items():
            symbol, sec_type, strike, right, expiry = key
            if sec_type == "OPT":
                group_key = (symbol, expiry, right)  # Group by symbol, expiry, and call/put
                if group_key not in option_groups:
                    option_groups[group_key] = []
                option_groups[group_key].append((strike, data))
        
        # Find vertical spreads in each group
        for group_key, options in option_groups.items():
            symbol, expiry, right = group_key
            
            # Need at least 2 options of same type to form a spread
            if len(options) >= 2:
                options.sort()  # Sort by strike price
                
                # Check each pair for opposite positions (long/short combination)
                for i in range(len(options)):
                    for j in range(i + 1, len(options)):
                        low_strike, low_data = options[i]
                        high_strike, high_data = options[j]
                        
                        low_pos = low_data['position']
                        high_pos = high_data['position']
                        
                        # Check for vertical spread patterns
                        if (low_pos > 0 and high_pos < 0) or (low_pos < 0 and high_pos > 0):
                            # Determine spread type
                            if right == "C":  # Call spread
                                if low_pos > 0 and high_pos < 0:
                                    spread_type = "Bull Call Spread"
                                else:
                                    spread_type = "Bear Call Spread"
                            else:  # Put spread
                                if low_pos < 0 and high_pos > 0:
                                    spread_type = "Bull Put Spread"
                                else:
                                    spread_type = "Bear Put Spread"
                            
                            spread_info = {
                                'symbol': symbol,
                                'expiry': expiry,
                                'type': spread_type,
                                'low_strike': low_strike,
                                'high_strike': high_strike,
                                'low_position': low_pos,
                                'high_position': high_pos,
                                'net_position': abs(low_pos) if abs(low_pos) <= abs(high_pos) else abs(high_pos),
                                'low_contract': low_data['contract'],
                                'high_contract': high_data['contract']
                            }
                            self.vertical_spreads.append(spread_info)
    
    def get_best_price(self, option_key):
        """Get the best available price for an option"""
        if option_key not in self.options_data:
            return None, "No data"
        
        price_data = self.options_data[option_key]
        
        # Try different price sources in order of preference
        if 'last' in price_data and price_data['last'] and price_data['last'] > 0:
            return price_data['last'], "Last"
        elif 'close' in price_data and price_data['close'] and price_data['close'] > 0:
            return price_data['close'], "Close"
        elif 'historical_close' in price_data and price_data['historical_close'] and price_data['historical_close'] > 0:
            return price_data['historical_close'], "Hist Close"
        elif 'bid' in price_data and 'ask' in price_data and price_data['bid'] and price_data['ask'] and price_data['bid'] > 0 and price_data['ask'] > 0:
            midpoint = (price_data['bid'] + price_data['ask']) / 2
            return midpoint, "Midpoint"
        elif 'bid' in price_data and price_data['bid'] and price_data['bid'] > 0:
            return price_data['bid'], "Bid"
        elif 'ask' in price_data and price_data['ask'] and price_data['ask'] > 0:
            return price_data['ask'], "Ask"
        
        return None, "No price"
    
    def get_positions_dataframe(self):
        """Return a pandas DataFrame with all position information"""
        if not self.positions:
            print(f"No positions found for account {self.target_account}.")
            return pd.DataFrame()
        
        data = []
        
        for key, position_data in self.positions.items():
            symbol, sec_type, strike, right, expiry = key
            contract = position_data['contract']
            position_size = position_data['position']  # Already converted to float
            avg_cost = position_data['avgCost']        # Already converted to float
            account = position_data['account']
            
            # Get current market price using improved method
            current_price, price_source = self.get_best_price(key)
            bid_price = None
            ask_price = None
            market_value = None
            unrealized_pnl = None
            
            if key in self.options_data:
                price_data = self.options_data[key]
                bid_price = price_data.get('bid')
                ask_price = price_data.get('ask')
                
                # Calculate market value and P&L
                if current_price is not None:
                    if sec_type == "OPT":
                        market_value = current_price * position_size * 100  # Options multiplier
                        cost_basis = avg_cost * position_size * 100
                    else:
                        market_value = current_price * position_size
                        cost_basis = avg_cost * position_size
                    
                    unrealized_pnl = market_value - cost_basis
            
            # Format expiry date if it's an option
            formatted_expiry = None
            if sec_type == "OPT" and expiry:
                try:
                    # Convert YYYYMMDD to YYYY-MM-DD
                    if len(expiry) == 8:
                        formatted_expiry = f"{expiry[:4]}-{expiry[4:6]}-{expiry[6:8]}"
                    else:
                        formatted_expiry = expiry
                except:
                    formatted_expiry = expiry
            
            row_data = {
                'Account': account,
                'Symbol': symbol,
                'SecType': sec_type,
                'Description': self._create_description(symbol, sec_type, strike, right, formatted_expiry),
                'Position': int(position_size),  # Convert to int for cleaner display
                'AvgCost': f"${avg_cost:.2f}" if avg_cost else "N/A",
                'CurrentPrice': f"${current_price:.2f}" if current_price else "N/A",
                'PriceSource': price_source,
                'Bid': f"${bid_price:.2f}" if bid_price else "N/A",
                'Ask': f"${ask_price:.2f}" if ask_price else "N/A",
                'MarketValue': f"${market_value:.2f}" if market_value else "N/A",
                'UnrealizedPnL': f"${unrealized_pnl:.2f}" if unrealized_pnl else "N/A",
                'Strike': strike if sec_type == "OPT" else None,
                'Right': right if sec_type == "OPT" else None,
                'Expiry': formatted_expiry if sec_type == "OPT" else None
            }
            
            data.append(row_data)
        
        # Create DataFrame
        df = pd.DataFrame(data)
        
        # Sort by symbol, security type, then strike/expiry for options
        if not df.empty:
            sort_columns = ['Symbol', 'SecType']
            if 'Strike' in df.columns:
                sort_columns.extend(['Strike', 'Expiry'])
            df = df.sort_values(sort_columns)
            df = df.reset_index(drop=True)
        
        return df
    
    def get_vertical_spreads_dataframe(self):
        """Return a pandas DataFrame with vertical spread information"""
        if not self.vertical_spreads:
            return pd.DataFrame()
        
        data = []
        
        for spread in self.vertical_spreads:
            symbol = spread['symbol']
            expiry = spread['expiry']
            spread_type = spread['type']
            low_strike = spread['low_strike']
            high_strike = spread['high_strike']
            low_pos = spread['low_position']
            high_pos = spread['high_position']
            net_position = spread['net_position']
            
            # Format expiry
            formatted_expiry = expiry
            if len(expiry) == 8:
                formatted_expiry = f"{expiry[:4]}-{expiry[4:6]}-{expiry[6:8]}"
            
            # Calculate spread value if we have price data
            spread_value = None
            low_key = (symbol, "OPT", low_strike, spread['low_contract'].right, expiry)
            high_key = (symbol, "OPT", high_strike, spread['high_contract'].right, expiry)
            
            low_price, low_source = self.get_best_price(low_key)
            high_price, high_source = self.get_best_price(high_key)
            
            if low_price and high_price:
                low_value = low_price * low_pos * 100
                high_value = high_price * high_pos * 100
                spread_value = low_value + high_value
            
            data.append({
                'Symbol': symbol,
                'Type': spread_type,
                'Expiry': formatted_expiry,
                'LowStrike': low_strike,
                'HighStrike': high_strike,
                'LowPos': int(low_pos),
                'HighPos': int(high_pos),
                'NetPos': int(net_position),
                'SpreadValue': f"${spread_value:.2f}" if spread_value else "N/A",
                'LowPriceSource': low_source if low_price else "N/A",
                'HighPriceSource': high_source if high_price else "N/A"
            })
        
        return pd.DataFrame(data)
    
    def _create_description(self, symbol, sec_type, strike, right, expiry):
        """Create a readable description for the position"""
        if sec_type == "OPT":
            return f"{symbol} {expiry} {strike} {right}"
        else:
            return symbol
    
    def request_account_summary(self):
        """Request account summary information"""
        tags = "TotalCashValue,NetLiquidation,UnrealizedPnL,RealizedPnL,GrossPositionValue"
        self.reqAccountSummary(9001, "All", tags)
    
    def get_account_summary(self):
        """Get account summary as a formatted string"""
        if not self.account_info:
            return "No account information available"
        
        summary = f"\n=== Account Summary for {self.target_account} ===\n"
        for tag, data in self.account_info.items():
            value = data['value']
            currency = data['currency']
            
            # Format monetary values
            try:
                if currency and tag in ['TotalCashValue', 'NetLiquidation', 'UnrealizedPnL', 'RealizedPnL', 'GrossPositionValue']:
                    formatted_value = f"{currency} {float(value):,.2f}"
                else:
                    formatted_value = value
            except:
                formatted_value = value
            
            summary += f"{tag}: {formatted_value}\n"
        
        return summary

In [33]:
# Configuration
ACCOUNT_ID = "DU9233079"
HOST = '127.0.0.1'
PORT = 4002  # Use 4002 for paper trading, 4001 for live trading
CLIENT_ID = random.randint(1000, 9999)

print(f"=== IBKR Position Retrieval for Account {ACCOUNT_ID} ===")
print(f"Connecting to IB Gateway at {HOST}:{PORT}")
print("Make sure IB Gateway is running and API connections are enabled.")
print("If using a paper account, port should be 4002.")
print("If using a live account, port should be 4001.")
print()

=== IBKR Position Retrieval for Account DU9233079 ===
Connecting to IB Gateway at 127.0.0.1:4002
Make sure IB Gateway is running and API connections are enabled.
If using a paper account, port should be 4002.
If using a live account, port should be 4001.



In [34]:
# Create and connect the app
app = IBKRPositionApp(target_account=ACCOUNT_ID)
app.connectTWS(HOST, PORT, CLIENT_ID)

# Check connection status
if app.isConnectedAndReady():
    print("✅ Successfully connected to IBKR")
else:
    print("❌ Failed to connect to IB Gateway")
    print("Please check:")
    print("1. IB Gateway is running")
    print("2. API connections are enabled in IB Gateway settings")
    print("3. You are using the correct port number")
    print("4. Your firewall is not blocking the connection")

Connecting to IB Gateway at 127.0.0.1:4002 with client ID: 8614
Error 1754338625441: 2104
Error 1754338625441: 2107
Error 1754338625442: 2158
✅ Successfully connected to IBKR


Position received: UNH (OPT), -1 @ $314.34
Position received: UNH (OPT), -1 @ $444.34
Position received: AAPL (OPT), -1 @ $307.37
Position received: UNH (OPT), 1 @ $310.66
Position received: UNH (OPT), 1 @ $185.66
Position received: AAPL (OPT), 1 @ $118.63
Position data complete. Found 6 positions for account DU9233079
Error 1754338628674: 2119
Error 1754338628878: 2104
Got last for ('UNH', 'OPT', 240.0, 'P', '20250815'): $5.85
Got close for ('UNH', 'OPT', 240.0, 'P', '20250815'): $8.15
Got last for ('AAPL', 'OPT', 195.0, 'P', '20250822'): $2.23
Got close for ('AAPL', 'OPT', 195.0, 'P', '20250822'): $2.89
Got last for ('UNH', 'OPT', 230.0, 'P', '20250822'): $3.65
Got close for ('UNH', 'OPT', 230.0, 'P', '20250822'): $5.32
Got last for ('UNH', 'OPT', 230.0, 'P', '20250815'): $2.35
Got close for ('UNH', 'OPT', 230.0, 'P', '20250815'): $4.13
Got last for ('AAPL', 'OPT', 185.0, 'P', '20250822'): $0.75
Got close for ('AAPL', 'OPT', 185.0, 'P', '20250822'): $1.08
Error 1754338634246: 10197


In [35]:
# Request position data
if app.isConnectedAndReady():
    print(f"Requesting positions for account {ACCOUNT_ID}...")
    
    # Request positions
    app.reqPositions()
    
    # Request account summary
    app.request_account_summary()
    
    # Wait for data with timeout
    timeout = 30  # seconds
    start_time = time.time()
    
    print("Waiting for position data", end="")
    while not app.position_end_received and time.time() - start_time < timeout:
        time.sleep(1)
        print(".", end="", flush=True)
    print()
    
    # Wait longer for market data and historical data fallbacks
    if len(app.positions) > 0:
        print("Waiting for market data and historical fallbacks...")
        time.sleep(15)  # Increased time to allow for historical data requests
        
        # Show progress on data collection
        total_positions = len(app.positions)
        positions_with_data = len([k for k in app.positions.keys() if k in app.options_data])
        print(f"Market data received for {positions_with_data}/{total_positions} positions")
    
    # Wait for account summary
    summary_timeout = 10
    start_time = time.time()
    while not app.account_summary_end_received and time.time() - start_time < summary_timeout:
        time.sleep(0.5)
    
    print(f"Data collection complete. Found {len(app.positions)} positions.")

Requesting positions for account DU9233079...
Waiting for position data.
Waiting for market data and historical fallbacks...
Market data received for 5/6 positions
Data collection complete. Found 6 positions.


In [36]:
# Display account summary
if app.isConnectedAndReady():
    print(app.get_account_summary())


=== Account Summary for DU9233079 ===
GrossPositionValue: GBP 1,666.63
NetLiquidation: GBP 25,563.91
TotalCashValue: GBP 26,162.06



In [37]:
# Display all positions
if app.isConnectedAndReady():
    positions_df = app.get_positions_dataframe()
    
    if not positions_df.empty:
        print(f"\n=== All Open Positions for Account {ACCOUNT_ID} ===")
        print(f"Total positions: {len(positions_df)}")
        print()
        display(positions_df)
    else:
        print(f"\nNo open positions found for account {ACCOUNT_ID}")


=== All Open Positions for Account DU9233079 ===
Total positions: 6



,Account,Symbol,SecType,Description,Position,AvgCost,CurrentPrice,PriceSource,Bid,Ask,MarketValue,UnrealizedPnL,Strike,Right,Expiry
0,DU9233079,AAPL,OPT,AAPL 2025-08-22 185.0 P,1,$118.63,$0.75,Last,N/A,N/A,$75.00,$-11788.15,185.0,P,2025-08-22
1,DU9233079,AAPL,OPT,AAPL 2025-08-22 195.0 P,-1,$307.37,$2.23,Last,N/A,N/A,$-223.00,$30513.58,195.0,P,2025-08-22
2,DU9233079,UNH,OPT,UNH 2025-08-15 230.0 P,1,$185.66,$2.35,Last,N/A,N/A,$235.00,$-18331.15,230.0,P,2025-08-15
3,DU9233079,UNH,OPT,UNH 2025-08-22 230.0 P,1,$310.66,$3.65,Last,N/A,N/A,$365.00,$-30701.15,230.0,P,2025-08-22
4,DU9233079,UNH,OPT,UNH 2025-08-15 240.0 P,-1,$314.34,$5.85,Last,N/A,N/A,$-585.00,$30848.58,240.0,P,2025-08-15
5,DU9233079,UNH,OPT,UNH 2025-08-22 240.0 P,-1,$444.34,N/A,No data,N/A,N/A,N/A,N/A,240.0,P,2025-08-22


In [38]:
# Display vertical spreads if any
if app.isConnectedAndReady() and len(app.positions) > 0:
    spreads_df = app.get_vertical_spreads_dataframe()
    
    if not spreads_df.empty:
        print(f"\n=== Vertical Spreads Identified ===")
        print(f"Total spreads: {len(spreads_df)}")
        print()
        display(spreads_df)
    else:
        print("\nNo vertical spreads identified in current positions.")


=== Vertical Spreads Identified ===
Total spreads: 3



,Symbol,Type,Expiry,LowStrike,HighStrike,LowPos,HighPos,NetPos,SpreadValue,LowPriceSource,HighPriceSource
0,UNH,Bear Put Spread,2025-08-15,230.0,240.0,1,-1,1,$-350.00,Last,Last
1,UNH,Bear Put Spread,2025-08-22,230.0,240.0,1,-1,1,N/A,Last,N/A
2,AAPL,Bear Put Spread,2025-08-22,185.0,195.0,1,-1,1,$-148.00,Last,Last


In [39]:
# Summary statistics
if app.isConnectedAndReady() and len(app.positions) > 0:
    print("\n=== Position Summary ===")
    
    # Count by security type
    sec_type_counts = {}
    total_market_value = 0
    total_unrealized_pnl = 0
    
    for key, pos_data in app.positions.items():
        symbol, sec_type, strike, right, expiry = key
        
        if sec_type not in sec_type_counts:
            sec_type_counts[sec_type] = 0
        sec_type_counts[sec_type] += 1
        
        # Calculate totals if we have market data
        if key in app.options_data:
            price_data = app.options_data[key]
            current_price = price_data.get('last') or price_data.get('close')
            if current_price:
                position_size = pos_data['position']
                avg_cost = pos_data['avgCost']
                
                if sec_type == "OPT":
                    market_value = current_price * position_size * 100
                    cost_basis = avg_cost * position_size * 100
                else:
                    market_value = current_price * position_size
                    cost_basis = avg_cost * position_size
                
                total_market_value += market_value
                total_unrealized_pnl += (market_value - cost_basis)
    
    print(f"Positions by security type:")
    for sec_type, count in sec_type_counts.items():
        print(f"  {sec_type}: {count}")
    
    if total_market_value != 0:
        print(f"\nTotal Market Value: ${total_market_value:,.2f}")
        print(f"Total Unrealized P&L: ${total_unrealized_pnl:,.2f}")
    
    if len(app.vertical_spreads) > 0:
        print(f"\nVertical Spreads: {len(app.vertical_spreads)}")


=== Position Summary ===
Positions by security type:
  OPT: 6

Total Market Value: $-133.00
Total Unrealized P&L: $541.71

Vertical Spreads: 3


In [40]:
# Disconnect from IB Gateway
if app.isConnected():
    app.disconnect()
    print("\n🔌 Disconnected from IB Gateway")
    
print("\n✅ Position retrieval complete!")


🔌 Disconnected from IB Gateway

✅ Position retrieval complete!
